In [1]:
!pip install pycryptodome


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.6 MB/s eta 0:00:00


In [2]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes

In [3]:
def aes_process(data, key, mode, operation, iv=None):
    if mode == AES.MODE_CBC:
        cipher = AES.new(key, mode, iv)
    else:
        cipher = AES.new(key, mode)

    if operation == "encrypt":
        ciphertext = cipher.encrypt(pad(data, AES.block_size))
        return ciphertext, cipher.iv if mode == AES.MODE_CBC else None
    elif operation == "decrypt":
        decrypted_data = unpad(cipher.decrypt(data), AES.block_size)
        return decrypted_data

In [4]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP

def rsa_process(data, key_components, operation):
    key = RSA.construct(key_components)
    cipher_rsa = PKCS1_OAEP.new(key)

    if operation == "encrypt":
        ciphertext = cipher_rsa.encrypt(data)
        return ciphertext
    elif operation == "decrypt":
        decrypted_data = cipher_rsa.decrypt(data)
        return decrypted_data

In [5]:
import hashlib

def hash_data(data, hash_mode):
    if hash_mode == "SHA1":
        hash_obj = hashlib.sha1()
    elif hash_mode == "SHA256":
        hash_obj = hashlib.sha256()
    elif hash_mode == "MD5":
        hash_obj = hashlib.md5()
    else:
        raise ValueError("Invalid hash mode. Use 'SHA1', 'SHA256', or 'MD5'.")

    hash_obj.update(data)
    hash_value = hash_obj.hexdigest()
    return hash_value

In [6]:
import random
from hashlib import sha256


def coprime(a, b):
    while b != 0:
        a, b = b, a % b
    return a

def modinv(a, m):
    return pow(a, -1, m)


def is_prime(num):
    if num == 2:
        return True
    if num < 2 or num % 2 == 0:
        return False
    for n in range(3, int(num**0.5)+2, 2):
        if num % n == 0:
            return False
    return True


def generate_keypair(p, q):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError('Both numbers must be prime.')
    elif p == q:
        raise ValueError('p and q cannot be equal')

    n = p * q
    phi = (p-1) * (q-1)
    e = random.randrange(1, phi)
    g = coprime(e, phi)

    while g != 1:
        e = random.randrange(1, phi)
        g = coprime(e, phi)

    d = modinv(e, phi)

    #Public key is (e, n) and private key is (d, n)
    return ((e, n), (d, n))


def encrypt(privatek, plaintext):
    key, n = privatek

    numberRepr = [ord(char) for char in plaintext]
    print("Number representation before encryption: ", numberRepr)
    cipher = [pow(ord(char),key,n) for char in plaintext]
    return cipher


def decrypt(publick, ciphertext):
    key, n = publick
    numberRepr = [pow(char, key, n) for char in ciphertext]
    plain = [chr(pow(char, key, n)) for char in ciphertext]
    print("Decrypted number representation is: ", numberRepr)
    return ''.join(plain)


def hashFunction(message):
    hashed = sha256(message.encode("UTF-8")).hexdigest()
    return hashed

def sign(private_key, message):
    hashed = hashFunction(message)
    return encrypt(private_key, hashed)

def verify_signature(public_key, signature, message):
    decrypted_hash = decrypt(public_key, signature)
    hashed = hashFunction(message)
    return decrypted_hash == hashed

In [7]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac
import base64

def generate_mac(data, key, algorithm):

    key = key.encode() if isinstance(key, str) else key

    if algorithm == "SHA256":
        hash_algorithm = hashes.SHA256()
    elif algorithm == "SHA512":
        hash_algorithm = hashes.SHA512()
    else:
        print("Invalid algorithm. Supported algorithms are SHA256 and SHA512.")
        return None
    h = hmac.HMAC(key, hash_algorithm, backend=default_backend())
    h.update(data)
    mac_value = h.finalize()

    return mac_value


In [10]:

while True:
    print("Choose system to implement:")
    print("1. AES Encryption")
    print("2. RSA Encryption")
    print("3. Hashing")
    print("4. Digital Signature")
    print("5. MAC Key Generation")
    print("6. Exit")

    choice = input("Enter your choice: ")

    if choice == "1":
        while True:
          key = input("Enter AES key: ").encode()
          while len(key) not in (16, 24, 32):
              print("AES key must be 16, 24, or 32 bytes in length.")
              key = input("Enter AES key: ").encode()

          operation = input("Enter operation (encrypt/decrypt/exit): ").lower()
          if operation == "exit":
              print("Exiting the program.")
              break
          elif operation not in ("encrypt", "decrypt"):
              print("Invalid operation. Use 'encrypt', 'decrypt', or 'exit'.")
          else:
              mode_str = input("Enter mode (ECB/CBC): ").upper()
              mode = AES.MODE_CBC if mode_str == "CBC" else AES.MODE_ECB

              if mode == AES.MODE_CBC:
                  iv = get_random_bytes(16)
                  print("IV:", iv.hex())
              else:
                  iv = None

              if operation == "encrypt":
                  plaintext = input("Enter plaintext: ").encode()
                  ciphertext, iv_used = aes_process(plaintext, key, mode, operation, iv)
                  print("Ciphertext:", ciphertext.hex())
              else:
                  ciphertext_hex = input("Enter ciphertext (in hex): ")
                  ciphertext = bytes.fromhex(ciphertext_hex)
                  decrypted_plaintext = aes_process(ciphertext, key, mode, operation, iv_used)
                  print("Decrypted plaintext:", decrypted_plaintext.decode())

    elif choice == "2":
        while True:
          operation = input("Enter operation (encrypt/decrypt/exit): ").lower()
          if operation == "exit":
              print("Exiting the program.")
              break
          elif operation not in ("encrypt", "decrypt"):
              print("Invalid operation. Use 'encrypt', 'decrypt', or 'exit'.")
          else:
              p = int(input("Enter RSA key component P: "))
              q = int(input("Enter RSA key component Q: "))
              e = int(input("Enter RSA key component e: "))  # Public
              d = int(input("Enter RSA key component d: "))  # Private
              n = p * q
              key_components = (n, e, d, p, q)

              if operation == "encrypt":
                  plaintext = input("Enter plaintext: ").encode()
                  ciphertext = rsa_process(plaintext, key_components, operation)
                  print("Ciphertext:", ciphertext.hex())
              else:
                  ciphertext_hex = input("Enter ciphertext (in hex): ")
                  ciphertext = bytes.fromhex(ciphertext_hex)
                  decrypted_plaintext = rsa_process(ciphertext, key_components, operation)
                  print("Decrypted plaintext:", decrypted_plaintext.decode())


    elif choice == "3":
        while True:
          plaintext = input("Enter plaintext: ").encode()
          hash_mode = input("Enter hash mode (SHA1/SHA256/MD5): ").upper()

          try:
              hash_result = hash_data(plaintext, hash_mode)
              print(f"Hash value ({hash_mode}): {hash_result}")
          except ValueError as e:
              print(e)

          another_operation = input("Do you want to perform another hashing operation? (yes/no): ").lower()
          if another_operation != "yes":
              print("Exiting the program.")
              break

    elif choice == "4":
        while True:
            operation = input("Choose operation (Generation/Sign/Verify/Exit): ").capitalize()

            if operation == "Generation":
                p = int(input("Enter a prime number: "))
                q = int(input("Enter another prime number: "))

                public, private = generate_keypair(p, q)
                print("Your public key is", public, "and your private key is", private)

            elif operation == "Sign":
                private_key = tuple(map(int, input("Enter the private key: ").split(',')))
                message = input("Enter the message to sign: ")

                signature = sign(private_key, message)
                print("Generated signature:", signature)

            elif operation == "Verify":
                public_key = tuple(map(int, input("Enter the public key: ").split(',')))
                signature = list(map(int, input("Enter the signature : ").split(',')))
                message = input("Enter the original message to verify: ")

                if verify_signature(public_key, signature, message):
                    print("Signature is valid.")
                else:
                    print("Signature is invalid.")

            elif operation == "Exit":
                break

            else:
                print("Invalid operation. Please choose Generation, Sign, Verify, or Exit.")

    elif choice == "5":
        while True:
          data = input("Enter data: ").encode()
          key = input("Enter key: ")
          algorithm = input("Enter MAC algorithm (SHA256/SHA512): ").upper()

          mac_value = generate_mac(data, key, algorithm)
          if mac_value:
              print(f"Generated {algorithm} MAC: {base64.b64encode(mac_value).decode()}\n")

          another = input("Do you want to generate another MAC? (yes/no): ").strip().lower()
          if another != "yes":
              break

    elif choice == "6":
        print("Exiting the program.")
        break

    else:
        print("Invalid choice. Please choose a valid option.")

Choose system to implement:
1. AES Encryption
2. RSA Encryption
3. Hashing
4. Digital Signature
5. MAC Key Generation
6. Exit
Enter your choice: 1
Enter AES key: 4739472345924565
Enter operation (encrypt/decrypt/exit): encrypt
Enter mode (ECB/CBC): CBC
IV: 8d7e532fd2bcbac402cdc5aad2dcecf8
Enter plaintext: Hello AES
Ciphertext: b3bf975bd14f85646c7bf972c5874310
Enter AES key: 4739472345924565
Enter operation (encrypt/decrypt/exit): decrypt
Enter mode (ECB/CBC): CBC
IV: 1fcf12e97fa09470065bf07eaf2fff09
Enter ciphertext (in hex): b3bf975bd14f85646c7bf972c5874310
Decrypted plaintext: Hello AES
Enter AES key: 4739472345924565
Enter operation (encrypt/decrypt/exit): exit
Exiting the program.
Choose system to implement:
1. AES Encryption
2. RSA Encryption
3. Hashing
4. Digital Signature
5. MAC Key Generation
6. Exit
Enter your choice: 6
Exiting the program.


In [ ]:
from Crypto.PublicKey import RSA

key = RSA.generate(1024)

print("RSA key components:")
print("P:", key.p)
print("Q:", key.q)
print("e:", key.e)
print("d:", key.d)

RSA key components:
P: 9534768163643759514646221491454725239356674954203125764269257457141984739816258353356809594673973811411377555558635314780706845692037428891952900356465171
Q: 12062028713180288484851652683547777874564777039679202877389246737870005466217631128137804782526904551754831928199803664929887681973785834625057173163559239
e: 65537
d: 10098668376039671926039165106643070265429677418418083417687910445718464455323059081643284323749846080051709476373159051215580342618317523398563097748595063692552442579510998854371685076525507369759774806779720421466100699829601290810883718425851757696609093346778370991433876549729420481345025619038310040513
